# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** The default looks reasonable, although I have some doubts as to whether it makes sense to discard the data around the edge in this case. 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

In [4]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

import nltk
nltk.download('punkt')

## TODO #1: Select appropriate values for the Python variables below.
batch_size=10
#batch_size = 64          # batch size, we have 10GB of GPU memory, let's use it
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = False    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 25             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, max_batch_size=batch_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) + list(encoder.bn.parameters())

# TODO #4: Define the optimizer.
#optimizer = torch.optim.SGD(params, lr=0.01)

# this data is probably pretty sparse, and defaults are probably ok
#http://ruder.io/optimizing-gradient-descent/
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to /home/sthenc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


loading annotations into memory...
Done (t=0.52s)
creating index...
index created!
[0/414113] Tokenizing captions...


 37%|███▋      | 151876/414113 [00:30<00:26, 9799.58it/s]

[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...
Done (t=0.52s)
creating index...



  0%|          | 943/414113 [00:00<00:43, 9428.68it/s]


index created!
Obtaining caption lengths...


 31%|███       | 127796/414113 [00:14<00:29, 9754.73it/s]


 62%|██████▏   | 255661/414113 [00:28<00:17, 9201.94it/s]


 92%|█████████▏| 382293/414113 [00:41<00:03, 9107.88it/s]


100%|██████████| 414113/414113 [00:45<00:00, 9143.49it/s]

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# temporary
%load_ext autoreload
%autoreload 2

from model import EncoderCNN, DecoderRNN

# Open the training log file.
f = open(log_file, 'w')

## running the training locally
#old_time = time.time()
#response = requests.request("GET", 
#                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
#                            headers={"Metadata-Flavor":"Google"})


for epoch in range(1, num_epochs+1):

    for i_step in range(1, total_step+1):
        
        ## running the training locally
        #if time.time() - old_time > 60:
        #    old_time = time.time()
        #    requests.request("POST", 
        #                     "https://nebula.udacity.com/api/v1/remote/keep-alive", 
        #                     headers={'Authorization': "STAR " + response.text})

        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler

        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)

        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()

        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)

        # Calculate the batch loss.
        loss = criterion(outputs.contiguous().view(-1, vocab_size), captions.contiguous().view(-1))

        # Backward pass.
        loss.backward(retain_graph=True)

        # Update the parameters in the optimizer.
        optimizer.step()

        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))

        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()

        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()

        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)

    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Epoch [1/25], Step [100/41412], Loss: 4.4867, Perplexity: 88.8297
Epoch [1/25], Step [200/41412], Loss: 4.0446, Perplexity: 57.08937
Epoch [1/25], Step [300/41412], Loss: 3.7327, Perplexity: 41.79326
Epoch [1/25], Step [400/41412], Loss: 4.2026, Perplexity: 66.8612
Epoch [1/25], Step [500/41412], Loss: 4.0259, Perplexity: 56.0289
Epoch [1/25], Step [600/41412], Loss: 3.3031, Perplexity: 27.19644
Epoch [1/25], Step [700/41412], Loss: 3.5537, Perplexity: 34.9435
Epoch [1/25], Step [800/41412], Loss: 3.8278, Perplexity: 45.9632
Epoch [1/25], Step [900/41412], Loss: 2.8911, Perplexity: 18.0137
Epoch [1/25], Step [1000/41412], Loss: 3.3596, Perplexity: 28.7786
Epoch [1/25], Step [1100/41412], Loss: 2.5576, Perplexity: 12.9045
Epoch [1/25], Step [1200/41412], Loss: 3.7884, Perplexity: 44.1878
Epoch [1/25], Step [1300/41412], Loss: 3.5841, Perplexity: 36.0227
Epoch [1/25], Step [1400/41412], Loss: 2.8287, 

Epoch [1/25], Step [12100/41412], Loss: 2.5063, Perplexity: 12.2601
Epoch [1/25], Step [12200/41412], Loss: 2.4971, Perplexity: 12.1468
Epoch [1/25], Step [12300/41412], Loss: 2.1686, Perplexity: 8.74586
Epoch [1/25], Step [12400/41412], Loss: 2.5789, Perplexity: 13.1820
Epoch [1/25], Step [12500/41412], Loss: 2.3958, Perplexity: 10.9772
Epoch [1/25], Step [12600/41412], Loss: 3.1249, Perplexity: 22.7570
Epoch [1/25], Step [12700/41412], Loss: 2.6593, Perplexity: 14.2860
Epoch [1/25], Step [12800/41412], Loss: 3.3241, Perplexity: 27.7746
Epoch [1/25], Step [12900/41412], Loss: 2.4258, Perplexity: 11.3112
Epoch [1/25], Step [13000/41412], Loss: 1.9556, Perplexity: 7.06794
Epoch [1/25], Step [13100/41412], Loss: 2.7004, Perplexity: 14.8857
Epoch [1/25], Step [13200/41412], Loss: 2.1548, Perplexity: 8.62640
Epoch [1/25], Step [13300/41412], Loss: 2.5604, Perplexity: 12.9410
Epoch [1/25], Step [13400/41412], Loss: 2.2975, Perplexity: 9.94906
Epoch [1/25], Step [13500/41412], Loss: 2.3957, 

Epoch [1/25], Step [24100/41412], Loss: 2.3175, Perplexity: 10.1506
Epoch [1/25], Step [24200/41412], Loss: 2.6511, Perplexity: 14.1691
Epoch [1/25], Step [24300/41412], Loss: 2.1793, Perplexity: 8.84007
Epoch [1/25], Step [24400/41412], Loss: 2.2747, Perplexity: 9.72490
Epoch [1/25], Step [24500/41412], Loss: 2.5413, Perplexity: 12.6961
Epoch [1/25], Step [24600/41412], Loss: 2.3133, Perplexity: 10.1077
Epoch [1/25], Step [24700/41412], Loss: 2.2860, Perplexity: 9.83538
Epoch [1/25], Step [24800/41412], Loss: 1.8260, Perplexity: 6.20927
Epoch [1/25], Step [24900/41412], Loss: 2.4143, Perplexity: 11.1818
Epoch [1/25], Step [25000/41412], Loss: 2.1111, Perplexity: 8.25732
Epoch [1/25], Step [25100/41412], Loss: 2.4265, Perplexity: 11.3192
Epoch [1/25], Step [25200/41412], Loss: 2.2929, Perplexity: 9.90333
Epoch [1/25], Step [25300/41412], Loss: 2.1054, Perplexity: 8.21081
Epoch [1/25], Step [25400/41412], Loss: 2.5464, Perplexity: 12.7612
Epoch [1/25], Step [25500/41412], Loss: 1.8714, 

Epoch [1/25], Step [36100/41412], Loss: 2.0179, Perplexity: 7.52243
Epoch [1/25], Step [36200/41412], Loss: 2.1142, Perplexity: 8.28262
Epoch [1/25], Step [36300/41412], Loss: 3.1995, Perplexity: 24.5212
Epoch [1/25], Step [36400/41412], Loss: 2.4477, Perplexity: 11.5618
Epoch [1/25], Step [36500/41412], Loss: 2.1331, Perplexity: 8.44097
Epoch [1/25], Step [36600/41412], Loss: 2.1254, Perplexity: 8.37616
Epoch [1/25], Step [36700/41412], Loss: 2.1860, Perplexity: 8.89994
Epoch [1/25], Step [36800/41412], Loss: 2.4551, Perplexity: 11.6472
Epoch [1/25], Step [36900/41412], Loss: 1.9135, Perplexity: 6.77717
Epoch [1/25], Step [37000/41412], Loss: 2.1792, Perplexity: 8.83909
Epoch [1/25], Step [37100/41412], Loss: 1.9393, Perplexity: 6.95381
Epoch [1/25], Step [37200/41412], Loss: 2.5538, Perplexity: 12.8554
Epoch [1/25], Step [37300/41412], Loss: 2.5879, Perplexity: 13.3020
Epoch [1/25], Step [37400/41412], Loss: 2.2268, Perplexity: 9.27055
Epoch [1/25], Step [37500/41412], Loss: 1.7550, 

Epoch [2/25], Step [6800/41412], Loss: 2.2374, Perplexity: 9.36911
Epoch [2/25], Step [6900/41412], Loss: 2.1276, Perplexity: 8.39455
Epoch [2/25], Step [7000/41412], Loss: 2.0877, Perplexity: 8.06643
Epoch [2/25], Step [7100/41412], Loss: 1.8404, Perplexity: 6.29929
Epoch [2/25], Step [7200/41412], Loss: 2.9338, Perplexity: 18.7980
Epoch [2/25], Step [7300/41412], Loss: 2.8647, Perplexity: 17.5431
Epoch [2/25], Step [7400/41412], Loss: 1.7883, Perplexity: 5.97938
Epoch [2/25], Step [7500/41412], Loss: 2.7257, Perplexity: 15.2664
Epoch [2/25], Step [7600/41412], Loss: 1.8025, Perplexity: 6.06469
Epoch [2/25], Step [7700/41412], Loss: 2.2752, Perplexity: 9.72961
Epoch [2/25], Step [7800/41412], Loss: 2.3196, Perplexity: 10.1715
Epoch [2/25], Step [7900/41412], Loss: 2.1648, Perplexity: 8.71249
Epoch [2/25], Step [8000/41412], Loss: 2.2196, Perplexity: 9.20357
Epoch [2/25], Step [8100/41412], Loss: 2.2325, Perplexity: 9.32281
Epoch [2/25], Step [8200/41412], Loss: 2.1108, Perplexity: 8.2

...

Epoch [10/25], Step [600/41412], Loss: 1.9670, Perplexity: 7.14940
Epoch [10/25], Step [700/41412], Loss: 1.9770, Perplexity: 7.22118
Epoch [10/25], Step [800/41412], Loss: 1.8443, Perplexity: 6.32397
Epoch [10/25], Step [900/41412], Loss: 1.7835, Perplexity: 5.95040
Epoch [10/25], Step [1000/41412], Loss: 1.7942, Perplexity: 6.0150
Epoch [10/25], Step [1100/41412], Loss: 1.7675, Perplexity: 5.85610
Epoch [10/25], Step [1200/41412], Loss: 2.0946, Perplexity: 8.12225
Epoch [10/25], Step [1300/41412], Loss: 1.7573, Perplexity: 5.79709
Epoch [10/25], Step [1400/41412], Loss: 2.0848, Perplexity: 8.04286
Epoch [10/25], Step [1500/41412], Loss: 1.8970, Perplexity: 6.66592
Epoch [10/25], Step [1600/41412], Loss: 2.3509, Perplexity: 10.4952
Epoch [10/25], Step [1700/41412], Loss: 2.0510, Perplexity: 7.77559
Epoch [10/25], Step [1800/41412], Loss: 2.2751, Perplexity: 9.72848
Epoch [10/25], Step [1900/41412], Loss: 2.0344, Perplexity: 7.647328
Epoch [10/25], Step [2000/41412], Loss: 2.1679, Perp

Epoch [10/25], Step [12600/41412], Loss: 1.7938, Perplexity: 6.01251
Epoch [10/25], Step [12700/41412], Loss: 1.8992, Perplexity: 6.68035
Epoch [10/25], Step [12800/41412], Loss: 1.8574, Perplexity: 6.40722
Epoch [10/25], Step [12900/41412], Loss: 1.7708, Perplexity: 5.87543
Epoch [10/25], Step [13000/41412], Loss: 2.0266, Perplexity: 7.58791
Epoch [10/25], Step [13100/41412], Loss: 2.2101, Perplexity: 9.11641
Epoch [10/25], Step [13200/41412], Loss: 1.3301, Perplexity: 3.78146
Epoch [10/25], Step [13300/41412], Loss: 1.6987, Perplexity: 5.46709
Epoch [10/25], Step [13400/41412], Loss: 1.6853, Perplexity: 5.39428
Epoch [10/25], Step [13500/41412], Loss: 1.6456, Perplexity: 5.18397
Epoch [10/25], Step [13600/41412], Loss: 1.6825, Perplexity: 5.37923
Epoch [10/25], Step [13700/41412], Loss: 2.8993, Perplexity: 18.1606
Epoch [10/25], Step [13800/41412], Loss: 2.1816, Perplexity: 8.86054
Epoch [10/25], Step [13900/41412], Loss: 1.9450, Perplexity: 6.99378
Epoch [10/25], Step [14000/41412],

Epoch [10/25], Step [24400/41412], Loss: 1.3255, Perplexity: 3.76428
Epoch [10/25], Step [24500/41412], Loss: 2.3992, Perplexity: 11.0145
Epoch [10/25], Step [24600/41412], Loss: 1.9068, Perplexity: 6.73178
Epoch [10/25], Step [24700/41412], Loss: 1.8210, Perplexity: 6.17780
Epoch [10/25], Step [24800/41412], Loss: 2.1071, Perplexity: 8.22479
Epoch [10/25], Step [24900/41412], Loss: 2.1996, Perplexity: 9.02189
Epoch [10/25], Step [25000/41412], Loss: 1.7319, Perplexity: 5.65129
Epoch [10/25], Step [25100/41412], Loss: 2.1333, Perplexity: 8.44302
Epoch [10/25], Step [25200/41412], Loss: 2.2520, Perplexity: 9.50657
Epoch [10/25], Step [25300/41412], Loss: 1.9006, Perplexity: 6.68971
Epoch [10/25], Step [25400/41412], Loss: 1.9699, Perplexity: 7.17024
Epoch [10/25], Step [25500/41412], Loss: 1.2702, Perplexity: 3.56158
Epoch [10/25], Step [25600/41412], Loss: 1.7339, Perplexity: 5.66291
Epoch [10/25], Step [25700/41412], Loss: 2.3507, Perplexity: 10.4925
Epoch [10/25], Step [25800/41412],

Epoch [10/25], Step [36200/41412], Loss: 1.8718, Perplexity: 6.50000
Epoch [10/25], Step [36300/41412], Loss: 1.9673, Perplexity: 7.15169
Epoch [10/25], Step [36400/41412], Loss: 1.7470, Perplexity: 5.73764
Epoch [10/25], Step [36500/41412], Loss: 1.6984, Perplexity: 5.46525
Epoch [10/25], Step [36600/41412], Loss: 1.8761, Perplexity: 6.52817
Epoch [10/25], Step [36700/41412], Loss: 2.2140, Perplexity: 9.15221
Epoch [10/25], Step [36800/41412], Loss: 1.9597, Perplexity: 7.09700
Epoch [10/25], Step [36900/41412], Loss: 1.9669, Perplexity: 7.14888
Epoch [10/25], Step [37000/41412], Loss: 2.1881, Perplexity: 8.91824
Epoch [10/25], Step [37100/41412], Loss: 1.9396, Perplexity: 6.95605
Epoch [10/25], Step [37200/41412], Loss: 1.9378, Perplexity: 6.94320
Epoch [10/25], Step [37300/41412], Loss: 1.6943, Perplexity: 5.44268
Epoch [10/25], Step [37400/41412], Loss: 1.8510, Perplexity: 6.36608
Epoch [10/25], Step [37500/41412], Loss: 2.0712, Perplexity: 7.93446
Epoch [10/25], Step [37600/41412],

Epoch [11/25], Step [6700/41412], Loss: 2.1347, Perplexity: 8.45416
Epoch [11/25], Step [6800/41412], Loss: 1.5167, Perplexity: 4.55742
Epoch [11/25], Step [6900/41412], Loss: 2.1700, Perplexity: 8.75836
Epoch [11/25], Step [7000/41412], Loss: 1.8018, Perplexity: 6.06031
Epoch [11/25], Step [7100/41412], Loss: 1.9919, Perplexity: 7.32956
Epoch [11/25], Step [7200/41412], Loss: 1.3145, Perplexity: 3.72301
Epoch [11/25], Step [7300/41412], Loss: 1.5699, Perplexity: 4.80642
Epoch [11/25], Step [7307/41412], Loss: 2.0314, Perplexity: 7.6245

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.